### Web Scraping The-Numbers Movie Budgets

In [41]:
MAX_PAGE = 5701
BASE_URL = "https://www.the-numbers.com"
BASE_LIST_URL = "https://www.the-numbers.com/movie/budgets/all/"

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import sqlite3

In [4]:
page = '1'
response = requests.get(BASE_LIST_URL + page)
response.status_code

200

In [5]:
print(response.text)

<!DOCTYPE html>
<html>
<head>
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async src="https://www.googletagmanager.com/gtag/js?id=UA-1343128-1"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-1343128-1');
</script>
<meta http-equiv="PICS-Label" content='(PICS-1.1 "https://www.icra.org/ratingsv02.html" l gen true for "https://www.the-numbers.com/" r (cb 1 lz 1 nz 1 oz 1 vz 1) "https://www.rsac.org/ratingsv01.html" l gen true for "https://www.the-numbers.com/" r (n 0 s 0 v 0 l 0))'>
<!--<meta http-equiv="Content-Type" content="text/html; charset=ISO-8859-1" >-->
<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
<meta name="format-detection" content="telephone=no">   <!-- for apple mobile --> 
<meta property="fb:admins" content="521546213" />


<meta name="viewport" content="initial-scale=1">
<meta name="description" content="The budget of (nearl

In [6]:
soup = BeautifulSoup(response.text)

In [13]:
tables = soup.find_all('table')

In [14]:
len(tables)

1

In [16]:
table = tables[0]

In [17]:
len(table.find_all('th'))

6

In [19]:
col_names = []
for th in table.find_all('th'):
    col_names.append(th.text)
    
col_names = [th.text for th in table.find_all('th')]

In [48]:
col_names[0] = 'idx'
col_names

['idx',
 'Release_Date',
 'Release_Date_URL',
 'Movie',
 'Movie_URL',
 'Production_Budget',
 'Domestic_Gross',
 'Worldwide_Gross']

In [22]:
rows = table.find_all('tr')

In [23]:
len(rows)

101

In [24]:
data = []
for row in rows[1:]:
    data.append((td.text for td in row.find_all('td')))

In [38]:
data = []
for row in rows[1:]:
    data_row = []
    tds = row.find_all('td')
    data_row.append(tds[0].text)
    data_row.append(tds[1].find('a').text)
    data_row.append(BASE_URL + tds[1].find('a')['href'])
    data_row.append(tds[2].find('a').text)
    data_row.append(BASE_URL + tds[2].find('a')['href'])
    data_row.append(tds[3].text)
    data_row.append(tds[4].text)
    data_row.append(tds[5].text)
#     print(*data_row, sep='\n')
    data.append(data_row)
print(len(data))

100


In [39]:
def process_page(soup):
    """
    Takes in a page from requests, outputs a list of lists (table format)
    """
    tables = soup.find_all('table')
    table = tables[0]
    rows = table.find_all('tr')
    data = []
    for row in rows[1:]:
        data_row = []
        tds = row.find_all('td')
        data_row.append(tds[0].text)
        data_row.append(tds[1].find('a').text)
        data_row.append(BASE_URL + tds[1].find('a')['href'])
        data_row.append(tds[2].find('a').text)
        data_row.append(BASE_URL + tds[2].find('a')['href'])
        data_row.append(tds[3].text)
        data_row.append(tds[4].text)
        data_row.append(tds[5].text)
        data.append(data_row)
    return data

In [42]:
from os.path import join
the_movies_data = []
for i in range(1, MAX_PAGE+1, 100):
    response = requests.get(join(BASE_LIST_URL, str(i)))
    soup = BeautifulSoup(response.text, 'html.parser')
    the_movies_data.extend(process_page(soup))
    
print(len(the_movies_data))

5785


In [43]:
col_names.insert(2, "Release_Date_URL")
col_names.insert(4, 'Movie_URL')


['index', 'Release Date', 'Release_Date_URL', 'Movie', 'Movie_URL', 'Production Budget', 'Domestic Gross', 'Worldwide Gross']


In [50]:
col_names = [c.replace(' ', '_') for c in col_names]
print(col_names)
column_names = ','.join([c + ' TEXT' for c in col_names])

['idx', 'Release_Date', 'Release_Date_URL', 'Movie', 'Movie_URL', 'Production_Budget', 'Domestic_Gross', 'Worldwide_Gross']


In [44]:
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

In [51]:
column_names = ','.join([c + ' TEXT' for c in col_names])
sql = """CREATE TABLE movies ({column_names})"""
cur.execute(sql.format(column_names=column_names))

In [52]:
cur.execute("SELECT * FROM movies").fetchall()

[]

In [53]:
insert_sql = """
INSERT INTO movies VALUES(?, ?, ?, ?, ?, ?, ?, ?);
"""

In [54]:
cur.execute(insert_sql, the_movies_data[0])

In [55]:
cur.execute("SELECT * FROM movies").fetchall()

[('1',
  'Dec 18, 2009',
  'https://www.the-numbers.com/box-office-chart/daily/2009/12/18',
  'Avatar',
  'https://www.the-numbers.com/movie/Avatar#tab=summary',
  '$425,000,000',
  '$760,507,625',
  '$2,776,345,279')]

In [56]:
for d in the_movies_data[1:]:
    cur.execute(insert_sql, d)

In [57]:
cur.execute("SELECT COUNT(*) FROM movies").fetchone()

(5785,)